In [2]:
import pandas as pd
import requests
import io

In [4]:
state_info = pd.read_csv('../source_data/state_info.csv', dtype = 'object')
state_info = state_info.set_index('fips')
dfs = []

for fips,row in state_info.iterrows():
    r = requests.get(f'https://fred.stlouisfed.org/graph/fredgraph.csv?id=LAUST{fips}0000000000003A')
    df = pd.read_csv(io.BytesIO(r.content))
    df['year'] = pd.to_datetime(df['DATE']).dt.year
    df['unemployment_pct'] = df[f'LAUST{fips}0000000000003A']
    df['state'] = row['state_po']
    df = df[['year', 'state', 'unemployment_pct']]
    dfs.append(df)

all_data = pd.concat(dfs, axis = 0, ignore_index = True)
all_data['unemployment_pct'] = all_data['unemployment_pct']/100
all_data['unemployment_pct_1'] = all_data['unemployment_pct'].diff(1)
all_data['unemployment_pct_4'] = all_data['unemployment_pct'].diff(4)
all_data.to_csv('../processed_data/1976-2023-unemployment.csv', index = False, header = True)